In [1]:
import sys
import requests
from dotenv import load_dotenv
from crypto import convertDataToJSON, pinJSONtoIPFS, initContract, w3, pinFiletoIPFS

In [2]:
smart_contract = initContract()

Contract Address 0x4C7D27a202Cca3b59c7a58320353845D0eB46d48


In [6]:
def registerArtwork():
    title = input("Enter a title for the artwork: ")
    description = input("Enter a description for the artwork: ")
    artists = input("Enter a comma seperated list of artists:")
    file_ipfs = input("Enter Image IPFS link:")
    
    message_object = {
            "title": title,
            "description": description,
            "artists": artists,
            "file_ipfs": file_ipfs,
    }

    return message_object

In [7]:
def postArtwork():
    message_object = registerArtwork() # Create New Message Object
    json_data = convertDataToJSON(message_object) # Convert message object to JSON
    message_uri = pinJSONtoIPFS(json_data) # Pin JSON to IPFS and get Hash
    
    tx_hash = smart_contract.functions.registerArt(str(message_uri)).transact(
        {"from": w3.eth.accounts[0]}
    )
    #logs = w3.eth.get_logs()
    receipt = w3.eth.waitForTransactionReceipt(tx_hash)   
    return receipt #logs

In [8]:
def getLatestArtwork():
    ipfs_hash = smart_contract.functions.bid(token_id).call()
    message = requests.get(f"https://cloudflare-ipfs.com/ipfs/{ipfs_hash}")
    return message.json()

In [9]:
def getArtworkByID(message_id):
    ipfs_hash = smart_contract.functions.highestBid(message_id).call()
    message = requests.get(f"https://cloudflare-ipfs.com/ipfs/{ipfs_hash}")
    return message.json() 

In [10]:
option = input("Would you like to get or post Artwork?")

if option == "post":
    postArtwork()

if option == "get": #get not functional refer to pullUri function
    token_id = input("Enter artwork ID to retrieve the artwork. Hit enter to get the latest.")

    if (token_id):
        print(getArtworkByID(int(token_id)))
    else:
        print(getLatestArtwork())

Would you like to get or post Artwork? post
Enter a title for the artwork:  Pretty Colors
Enter a description for the artwork:  Expanded original image
Enter a comma seperated list of artists: me, amila and etc
Enter Image IPFS link: ipfs://bafybeia54ofch35rdwxtgvh465ih67mfteg7tofmvyumjazsdk7caor72e


bafkreihpp6ltezf7tjgoj2dswsrzjoo3tt3dyqvgfqrpxcxlvu6wjj4b2a


In [11]:
def auctionCreate(token_id):
    tx_hash = smart_contract.functions.createAuction(token_id).transact(
        {"from": w3.eth.accounts[0]}
    )
    receipt = w3.eth.waitForTransactionReceipt(tx_hash)   #waitFor
    return receipt #logs

In [12]:
auctionCreate(1)

AttributeDict({'transactionHash': HexBytes('0xbd32dbb0084ec2bad074432cb767bf1ce3e4fad45b93fd7ada9735796012e363'),
 'transactionIndex': 0,
 'blockHash': HexBytes('0x15bff0ebe0a151a560448cc506a76f20c14f21cec1d3970ab8d27f9ac0aa552c'),
 'blockNumber': 82,
 'from': '0xbA58F51e201390af6F13AcA36b09c58A59Df7006',
 'to': '0x4C7D27a202Cca3b59c7a58320353845D0eB46d48',
 'gasUsed': 562395,
 'cumulativeGasUsed': 562395,
 'contractAddress': None,
 'logs': [],
 'status': 1,
 'logsBloom': HexBytes('0x0000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000

In [13]:
def placeBid(token_id):
    tx_hash = smart_contract.functions.bid(token_id).transact(
    {"from": w3.eth.accounts[0],
     'value': 1000000 #Must hardcode bid value
    })
    receipt = w3.eth.waitForTransactionReceipt(tx_hash)
    return receipt

In [14]:
placeBid(1)

AttributeDict({'transactionHash': HexBytes('0x75c2a1ca42c420a253b8bcc8634a95c0f2d2ac1b1f2c274fcfa03eeb7770e25b'),
 'transactionIndex': 0,
 'blockHash': HexBytes('0x94819e9e75baaf719fa2727f662bfbe431269d2c35a65c427a11bbe3e59d6aec'),
 'blockNumber': 83,
 'from': '0xbA58F51e201390af6F13AcA36b09c58A59Df7006',
 'to': '0x4C7D27a202Cca3b59c7a58320353845D0eB46d48',
 'gasUsed': 76693,
 'cumulativeGasUsed': 76693,
 'contractAddress': None,
 'logs': [AttributeDict({'logIndex': 0,
   'transactionIndex': 0,
   'transactionHash': HexBytes('0x75c2a1ca42c420a253b8bcc8634a95c0f2d2ac1b1f2c274fcfa03eeb7770e25b'),
   'blockHash': HexBytes('0x94819e9e75baaf719fa2727f662bfbe431269d2c35a65c427a11bbe3e59d6aec'),
   'blockNumber': 83,
   'address': '0x6B62A835a2B17194B396bFeaE9E8cc8D2f2A23D4',
   'data': '0x000000000000000000000000ba58f51e201390af6f13aca36b09c58a59df700600000000000000000000000000000000000000000000000000000000000f4240',
   'topics': [HexBytes('0xf4757a49b326036464bec6fe419a4ae38c8a02ce3e68bf080

In [15]:
def closeAuction(token_id):
    tx_hash = smart_contract.functions.endAuction(token_id).transact({"from": w3.eth.accounts[0]})
    receipt = w3.eth.waitForTransactionReceipt(tx_hash)
    return receipt

In [16]:
closeAuction(1)

AttributeDict({'transactionHash': HexBytes('0x3895c4353180e31031a60632b7196d9daf722a1e55f6d4e0d68058cfbc229945'),
 'transactionIndex': 0,
 'blockHash': HexBytes('0xbaed3288766a3dda7cad1f8c881877b948fe9f0f4a4f5454c513745bd9a5a353'),
 'blockNumber': 84,
 'from': '0xbA58F51e201390af6F13AcA36b09c58A59Df7006',
 'to': '0x4C7D27a202Cca3b59c7a58320353845D0eB46d48',
 'gasUsed': 87290,
 'cumulativeGasUsed': 87290,
 'contractAddress': None,
 'logs': [AttributeDict({'logIndex': 0,
   'transactionIndex': 0,
   'transactionHash': HexBytes('0x3895c4353180e31031a60632b7196d9daf722a1e55f6d4e0d68058cfbc229945'),
   'blockHash': HexBytes('0xbaed3288766a3dda7cad1f8c881877b948fe9f0f4a4f5454c513745bd9a5a353'),
   'blockNumber': 84,
   'address': '0x6B62A835a2B17194B396bFeaE9E8cc8D2f2A23D4',
   'data': '0x000000000000000000000000ba58f51e201390af6f13aca36b09c58a59df700600000000000000000000000000000000000000000000000000000000000f4240',
   'topics': [HexBytes('0xdaec4582d5d9595688c8c98545fdd1c696d41c6aeaeb63673

In [16]:
# Not functional due to openZeppelin having multiple safeTransferFrom
#def transfer(token_id):
   # tx_hash = smart_contract.functions.safeTransferFrom(token_id).transact(
    #{"from": w3.eth.accounts[0],
     #'to': w3.eth.accounts[1]
#Must hardcode bid value
    #})
    #receipt = w3.eth.waitForTransactionReceipt(tx_hash)
    #return receipt

In [17]:
def pullUri(token_id):
    tx_hash = smart_contract.functions.tokenURI(token_id).call()
    receipt = w3.eth.waitForTransactionReceipt(tx_hash)
    return receipt

In [18]:
pullUri(1)

ValueError: when sending a str, it must be a hex string. Got: 'bafkreihpp6ltezf7tjgoj2dswsrzjoo3tt3dyqvgfqrpxcxlvu6wjj4b2a'